<a href="https://colab.research.google.com/github/Tejshah740/AI-ML_tutor_chatbot/blob/main/gpt_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

In [ ]:

tokenizer=T5Tokenizer.from_pretrained("google/flan-t5-small")
model=T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [ ]:
model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
with open("/content/sample_data/flan_t5_ml_tutor_v2.txt","r",encoding="utf-8") as f:
  text=f.readlines()

In [ ]:
from datasets import Dataset
questions=[]
answers=[]
for i in range(0,len(text),3):
  q=text[i].replace("question","").strip()
  a=text[i+1].replace("answer","").strip()
  questions.append(q)
  answers.append(a)

dataset=Dataset.from_dict({"questions":questions,"answers":answers})

In [ ]:
def preprocess(batch):
  input=tokenizer(
      batch["questions"],
      return_tensors='pt',
      padding=True,
      truncation=True
  )
  output=tokenizer(
      batch["answers"],
      return_tensors="pt",
      padding=True,
      truncation=True
  )
  input["labels"]=output["input_ids"]
  return input
tokenized_dataset=dataset.map(preprocess,batched=True)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
from transformers import TrainingArguments
training_args=TrainingArguments(
    output_dir="./flan-t5-ml-tutor",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=8,
    learning_rate=2e-4,
    weight_decay=0.01,
    save_total_limit=2,
    save_steps=500,
    report_to="none",
    remove_unused_columns=False
)

In [ ]:
from transformers import Trainer
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./flan-t5-ml-tutor")
tokenizer.save_pretrained("./flan-t5-ml-tutor")

In [ ]:
import torch
tokenizer = T5Tokenizer.from_pretrained("./flan-t5-ml-tutor")
model = T5ForConditionalGeneration.from_pretrained("./flan-t5-ml-tutor")
model.eval()
model.to("cuda")

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

In [ ]:
while(True):
  question=input("enter your question:")
  if(question.lower() in ["exit","quit"]):
    break
  prompt=f"question: {question} answer:"
  inputs=tokenizer(prompt,return_tensors="pt",padding=True)
  input_ids=inputs["input_ids"].to("cuda")
  attention_masks=inputs["attention_mask"].to("cuda")
  with torch.no_grad():
    output_ids=model.generate(input_ids=input_ids,
                              max_new_tokens=100,
                              temperature=0.3,
                              top_p=0.9,
                              repetition_penalty=1.2,
                              pad_token_id=tokenizer.eos_token_id,
                              do_sample=False,
                              attention_mask=attention_masks
                              )
  response=tokenizer.decode(output_ids[0],skip_special_tokens=True)
  print(response.replace("<pad>",""))


enter your question:what is underfitting
 Underfitting is when a model is too simple to capture patterns in the data.
enter your question:what is overfitting
 Overfitting is when a model learns training data too well and performs poorly on unseen data.
enter your question:what is gradient descent
 gradient descent is an optimization method that minimizes loss by updating parameters.
enter your question:what is backpropogation
 Backpropogation computes gradients of the loss with respect to model parameters.
enter your question:what is self attention
 Self attention allows a model to weigh relationships between tokens in a sequence.
enter your question:exit
